In [4]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
postgres_uri_mp = os.environ["POSTGRESQL_URI_MP"]

In [3]:
df = pd.read_parquet('/Users/larsankile/Downloads/openethdata_eth_data.parquet.gzip').sort_values('block_number')

# Filter out all transactions that are 'missed'
df = df[df.proposer_pubkey != 'missed']
df = df[~df.block_number.isna()]
df = df.astype({'block_number': int})

print(df.shape)

df.head()

(2834350, 9)


,date,slot,block_number,relay,builder_pubkey,proposer_pubkey,mevboost_value,builder,validator
923491,2022-09-15 06:42:59,4700013,15537394,None,None,0xa04520c5c37237e358cdd10d75d055d2e81ada151e63...,None,None,celsius
531576,2022-09-15 06:43:11,4700014,15537395,None,None,0xa605e658a0dd0dd77adb9fc2398f78f24c648125126d...,None,None,stakehound
667510,2022-09-15 06:43:23,4700015,15537396,None,None,0xb11a32a0f8f7acb81f959e7c68af49705bb64d78b41b...,None,None,lido
282876,2022-09-15 06:43:35,4700016,15537397,None,None,0x863b410dfb618b4f6b947f3ede710001aa274bc74b1d...,None,None,binance
215777,2022-09-15 06:43:47,4700017,15537398,None,None,0xb9be1d7a0a871eef07ca47796842b0449032462d69d2...,None,None,0xb9be1d7a0a871eef07ca47796842b0449032462d69d2...


In [8]:
df.to_sql('mev_boost', postgres_uri_mp, if_exists='replace', index=False, method='multi', chunksize=10_000)

In [25]:
min_block, max_block = df.block_number.min(), df.block_number.max()

min_block, max_block

(15537394, 17960034)

In [28]:
df[df.relay.isnull()].shape[0] / df.shape[0]

0.13439060101963413

In [29]:
all_blocks = set(range(min_block, max_block + 1))
not_boost = all_blocks - set(df.block_number.unique())

len(not_boost) / len(all_blocks)

0.00012176793837799327

In [19]:
df.builder_pubkey.isna().sum() / df.shape[0]

0.1343924334134456

In [20]:
df.proposer_pubkey.isna().sum() / df.shape[0]

0.0

In [21]:
df.date.max()

Timestamp('2023-08-21 02:09:35')